In [ ]:
# Installer les packages si ce n'est pas déjà fait
%pip install openai
%pip install python-dotenv

In [ ]:
import openai
import os
from dotenv import load_dotenv
import json
import random
import re


Chargement de l’API Key

In [ ]:
# Charger la clé OpenAI depuis le fichier .env
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set.")


In [ ]:
#Validation d’un format numérique
def is_valid_number_format(value):
    pattern = r'^-?\d+(\.\d+)?$'
    return bool(re.match(pattern, value))

#Charger la liste de mots
word_list = []

def load_word_list():
    with open("words_alpha.txt", "r") as file:
        for line in file:
            word_list.append(line.strip())
    return word_list

word_list = load_word_list()


Définir la dataclass WordleConstraint

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, Set

@dataclass
class WordleConstraints:
    green: Dict[int, str] = field(default_factory=dict)
    yellow: Dict[int, str] = field(default_factory=dict)
    grey: Set[str] = field(default_factory=set)


StrategyAgent


In [ ]:
class StrategyAgent:
    def __init__(self, model_name, api_key):
        self.model_name = model_name
        self.client = openai.OpenAI(api_key=api_key)

    def evaluate_suggestion(self, suggestion, constraints, past_guesses, word_length, threshold, context_message):
        """
        Utilise GPT pour évaluer si le mot proposé est correct.
        Retourne un score entre 0 (mauvais) et 1 (parfait) ou une meilleure suggestion.
        """
        constraints_serializable = {
            'green': constraints.get('green', {}),
            'yellow': list(constraints.get('yellow', [])),
            'grey': list(constraints.get('grey', []))
        }
        
        prompt = f"""
            You are an AI evaluator for Wordle strategies. Your task is to assess a suggested word based on the following criteria:
            - Word Length: {word_length}
            - Previous Guesses: {', '.join(past_guesses)}
            - Constraints:
              - Green: Letters in the correct positions.
              - Yellow: Letters in the word but in the wrong positions.
              - Grey: Letters that do not appear in the word at all.

            Evaluate the following suggestion: {suggestion}.

            Take into account the following context of the Wordle game: 
            {context_message}

            If the score you would give is less than the threshold of {threshold}, provide a better suggestion. 
            If the score is >= threshold, return the score as a number.
            Return ONLY the score or the better suggestion, nothing else.
        """

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": prompt},
                    {"role": "user", "content": f"Suggestion: {suggestion}"},
                    {"role": "user", "content": f"Constraints: {json.dumps(constraints_serializable)}"}
                ]
            )

            score_str = response.choices[0].message.content.strip()
            return score_str
        except Exception as e:
            print(f"Error evaluating suggestion: {e}")
            return 0.5
